In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.ticker import EngFormatter

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

In [ ]:
def get_topology(path):
    if 'a2a' in path:
        return "All-to-all"
    elif 'group5' in path:
        return "Groups of 5"

def read_data(csv_path):
    data = pd.read_csv(csv_path)
    
    metadata = {
        'duration': np.max(data.time),
        'worker_count': len(np.unique(data.worker)),
        'topology': get_topology(csv_path)
    }
    
    return data, metadata

In [ ]:
def plot_aggregate(data, metadata):
    d = data.groupby(['worker', 'time']).sum().diff().fillna(0).reset_index()
    
    # average throughput for [T/2, T]
    x = d[(d.time <= metadata['duration']) & (d.time >= metadata['duration'] / 2)]
    x = x.groupby('worker').mean()

    plt.scatter(8 * x.bytes_sent, 8 * x.bytes_received)
    plt.xlabel("Send Rate (bps)")
    plt.ylabel("Receive Rate (bps)")
    plt.title(f"Average throughput for {metadata['worker_count']} workers ({metadata['topology']})")
    plt.gca().xaxis.set_major_formatter(EngFormatter(unit=''))
    plt.gca().yaxis.set_major_formatter(EngFormatter(unit=''))

In [ ]:
data, metadata = read_data("./results/lambda-a2a-w160-30s.csv")
plot_aggregate(data, metadata)

In [ ]:
data, metadata = read_data("./results/lambda-group5-w160-30s.csv")
plot_aggregate(data, metadata)